In [1]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
TF_ENABLE_ONEDNN_OPTS = 0
URLS = CHUNKS_URLS
LIMIT = 60
LARGER_DATASET_PATH = "../larger_dataset"
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
ACTUAL_DATA_SHAPE = 6990280
RANDOM_STATE = 42

In [3]:
def get_chunks(urls,limit=0,verbose = 1,base_name = "",file_path="",file_format='.csv'):
    #downloads all data from their urls
    for i,url in enumerate(urls):
        if limit:
            if i == limit:
                return
        file_name = base_name + str(i + 1)
        #checks if file already exists
        if os.path.exists(file_path + file_name + file_format):
            print(f"{file_name} already exists.")
            continue
        if i % verbose == 0:
            print(f"Downloading {file_name}...")
        r = requests.get(url)
        with open(file_path + file_name + file_format, 'wb') as fd:
            for chunk in r.iter_content():
                #save file in the current directory of the notebook
                fd.write(chunk)

In [4]:
def get_all_file_names(base_name,limit_num):
    return [base_name + str(num) for num in range(1,limit_num + 1)]

In [5]:
def read_chunks(files,file_path = "",file_format = ".csv"):
    #reads chunks csvs and converts them to a dataframe format
    final_df = []
    for file in files:
        df = pd.read_csv(file_path + file + file_format)
        final_df.append(df)
    #function returns a list of dfs
    return final_df

In [6]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [7]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [8]:
def group_up_chunks(dfs):
    #adds up all dataframes together
    return pd.concat(dfs)

In [9]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [10]:
review_data = df[['text', 'stars']].copy()

In [11]:
review_data.reset_index(inplace = True)
review_data.drop(['index'],axis = 1,inplace = True)

In [12]:
review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [13]:
review_data.isnull().sum()
review_data.dropna()

,full_review_text,star_rating
0,decide eat aware go take 2 hour begin end try ...,3.0
1,ve take lot spin class year nothing compare cl...,5.0
2,family diner buffet eclectic assortment large ...,3.0
3,wow yummy different delicious favorite lamb cu...,5.0
4,cute interior owner give u tour upcoming patio...,4.0
...,...,...
6990275,late addition service iccu apple pay iccu debi...,5.0
6990276,spot offer great affordable east weekend paddl...,5.0
6990277,home depot need get lot demential lumber seem ...,4.0
6990278,m feel like ignore caloriecounting indulge fla...,5.0


In [14]:
X = review_data['full_review_text']
y = review_data['star_rating']

In [15]:
X = pd.Series([str(text) for text in X])

In [16]:
review_data.isnull().sum()

full_review_text    44
star_rating          0
dtype: int64

In [17]:
review_data.dropna()

,full_review_text,star_rating
0,decide eat aware go take 2 hour begin end try ...,3.0
1,ve take lot spin class year nothing compare cl...,5.0
2,family diner buffet eclectic assortment large ...,3.0
3,wow yummy different delicious favorite lamb cu...,5.0
4,cute interior owner give u tour upcoming patio...,4.0
...,...,...
6990275,late addition service iccu apple pay iccu debi...,5.0
6990276,spot offer great affordable east weekend paddl...,5.0
6990277,home depot need get lot demential lumber seem ...,4.0
6990278,m feel like ignore caloriecounting indulge fla...,5.0


In [18]:
if not os.path.exists('../pickle_files'):
    os.mkdir('../pickle_files')

In [19]:
if not os.path.exists("../pickle_files/word2vec_model_sklearn.pickle"):
    print("Creating Embedding From Scratch.")
    count_model = CountVectorizer()
    word2vec_model_sklearn = count_model.fit_transform(X.apply(lambda x: np.str_(x)))
    pickle_out = open("../pickle_files/word2vec_model_sklearn.pickle",'wb')
    pickle.dump(word2vec_model_sklearn,pickle_out)
    pickle_out.close()
else:
    print("Found Pickle File.")
    word2vec_model_sklearn = pickle.load(open("../pickle_files/word2vec_model_sklearn.pickle",'rb'))

Found Pickle File.


In [20]:
if not os.path.exists("../pickle_files/word2vec_model_gensim.model"):
    print("Creating Embedding From Scratch.")
    word2vec_model_gensim = Word2Vec(X, min_count=1, vector_size=100)
    word2vec_model_gensim.save('../pickle_files/word2vec_model_gensim.model')
else:
    print("Found Embedding File.")
    word2vec_model_gensim = Word2Vec.load("../pickle_files/word2vec_model_gensim.model")

Creating Embedding From Scratch.


In [22]:
if not os.path.exists("../pickle_files/svd_model_sklearn.pickle"):
    print("Creating SVD model From Scratch.")
    svd_model = TruncatedSVD(n_components=100, random_state=RANDOM_STATE)
    svd_model_sklearn = svd_model.fit_transform(word2vec_model_sklearn)
    word_vectors_sklearn = dict(zip(count_model.get_feature_names(), svd_model.components_.transpose()))
    pickle_out = open("../pickle_files/svd_model_sklearn.pickle",'wb')
    pickle.dump(word2vec_model_sklearn,pickle_out)
    pickle_out.close()
else:
    print("Found Pickle File.")
    word2vec_model_sklearn = pickle.load(open("../pickle_files/svd_model_sklearn.pickle",'rb'))

Creating SVD model From Scratch.


MemoryError: Unable to allocate 5.73 GiB for an array with shape (110, 6990280) and data type float64

In [ ]:
word1 = 'sentiment'
word2 = 'sarcasm'

In [ ]:
# Cosine similarity using scikit-learn model
vec1_sklearn = word_vectors_sklearn[word1]
vec2_sklearn = word_vectors_sklearn[word2]
similarity_sklearn = np.dot(vec1_sklearn, vec2_sklearn) / (np.linalg.norm(vec1_sklearn) * np.linalg.norm(vec2_sklearn))
print(f"Similarity between '{word1}' and '{word2}' using scikit-learn model: {similarity_sklearn:.4f}")

# Cosine similarity using gensim model
similarity_gensim = word2vec_model_gensim.wv.similarity(word1, word2)
print(f"Similarity between '{word1}' and '{word2}' using gensim model: {similarity_gensim:.4f}")

In [ ]:
word2vec_model_sklearn_array = word2vec_model_sklearn.toarray()

In [ ]:
x_train_sklearn,x_test_sklearn,y_train_sklearn,y_test_sklearn = create_train_test_split(word2vec_model_sklearn_array,y,test_size=0.2)
x_train_gensim,x_test_gensim,y_train_gensim,y_test_gensim = create_train_test_split(word2vec_model_gensim,y,test_size=0.2)

In [ ]:
smote_resampler = SMOTE(random_state=RANDOM_STATE)
x_train_sklearn,y_train_sklearn = smote_resampler.fit_resample(x_train_sklearn,y_train_sklearn)
x_train_gensim,y_train_gensim = smote_resampler.fit_resample(x_train_gensim,y_train_gensim)

In [ ]:
rf_clf_sklearn = RandomForestClassifier(n_estimators=100,random_state=RANDOM_STATE)
rf_clf_gensim = RandomForestClassifier(n_estimators=100,random_state=RANDOM_STATE)

In [ ]:
rf_clf_sklearn.fit(x_train_sklearn,y_train_sklearn)
rf_clf_gensim.fit(x_train_gensim,y_train_gensim)

y_pred_sklearn = rf_clf_sklearn.predict(x_test_sklearn)
y_pred_gensim = rf_clf_gensim.predict(x_test_gensim)

In [ ]:
print(f" Sklearn Embedding with RF Accuracy Score: {metrics.accuracy_score(y_test_sklearn,y_pred_sklearn)}")
print(f" Gensim Embedding with RF Accuracy Score: {metrics.accuracy_score(y_test_gensim,y_pred_gensim)}")
print(f" Sklearn Embedding with RF Accuracy Score: {metrics.f1_score(y_test_sklearn,y_pred_sklearn)}")
print(f" Gensim Embedding with RF Accuracy Score: {metrics.f1_score(y_test_gensim,y_pred_gensim)}")

In [ ]:
show_learning_curve(rf_clf_sklearn,word2vec_model_sklearn_array,y,5)
print('\n')
show_learning_curve(rf_clf_gensim,word2vec_model_gensim,y,5)